# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 5 2022 9:57AM,247923,15,8608276,"Brookfield Pharmaceuticals, LLC",Released
1,Oct 5 2022 9:57AM,247923,15,8608277,"Brookfield Pharmaceuticals, LLC",Released
2,Oct 5 2022 9:57AM,247923,15,8608278,"Brookfield Pharmaceuticals, LLC",Released
3,Oct 5 2022 9:57AM,247923,15,8608279,"Brookfield Pharmaceuticals, LLC",Released
4,Oct 5 2022 9:57AM,247923,15,8608280,"Brookfield Pharmaceuticals, LLC",Released
5,Oct 5 2022 9:57AM,247923,15,8608281,"Brookfield Pharmaceuticals, LLC",Released
6,Oct 5 2022 9:57AM,247923,15,8608282,"Brookfield Pharmaceuticals, LLC",Released
7,Oct 5 2022 9:57AM,247923,15,8608283,"Brookfield Pharmaceuticals, LLC",Released
8,Oct 5 2022 9:57AM,247923,15,8608284,"Brookfield Pharmaceuticals, LLC",Released
9,Oct 5 2022 9:57AM,247923,15,8608285,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,247917,Released,7
37,247918,Released,1
38,247919,Released,21
39,247921,Executing,1
40,247923,Released,17


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247917,NaN,NaN,7.0
247918,NaN,NaN,1.0
247919,NaN,NaN,21.0
247921,NaN,1.0,NaN
247923,NaN,NaN,17.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247677,0.0,0.0,1.0
247758,0.0,0.0,1.0
247760,0.0,0.0,1.0
247802,0.0,0.0,1.0
247841,12.0,0.0,49.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247677,0,0,1
247758,0,0,1
247760,0,0,1
247802,0,0,1
247841,12,0,49


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247677,0,0,1
1,247758,0,0,1
2,247760,0,0,1
3,247802,0,0,1
4,247841,12,0,49


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247677,,,1
1,247758,,,1
2,247760,,,1
3,247802,,,1
4,247841,12,,49


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 5 2022 9:57AM,247923,15,"Brookfield Pharmaceuticals, LLC"
17,Oct 5 2022 9:44AM,247898,21,"NBTY Global, Inc."
18,Oct 5 2022 9:44AM,247760,21,"NBTY Global, Inc."
19,Oct 5 2022 9:43AM,247903,21,"NBTY Global, Inc."
20,Oct 5 2022 9:43AM,247677,21,"NBTY Global, Inc."
21,Oct 5 2022 9:43AM,247921,12,Hush Hush
22,Oct 5 2022 9:42AM,247802,21,"NBTY Global, Inc."
23,Oct 5 2022 9:42AM,247758,21,"NBTY Global, Inc."
24,Oct 5 2022 9:30AM,247891,16,Sartorius Bioprocess Solutions
27,Oct 5 2022 9:12AM,247919,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 5 2022 9:57AM,247923,15,"Brookfield Pharmaceuticals, LLC",,,17
1,Oct 5 2022 9:44AM,247898,21,"NBTY Global, Inc.",,,1
2,Oct 5 2022 9:44AM,247760,21,"NBTY Global, Inc.",,,1
3,Oct 5 2022 9:43AM,247903,21,"NBTY Global, Inc.",,,1
4,Oct 5 2022 9:43AM,247677,21,"NBTY Global, Inc.",,,1
5,Oct 5 2022 9:43AM,247921,12,Hush Hush,,1,
6,Oct 5 2022 9:42AM,247802,21,"NBTY Global, Inc.",,,1
7,Oct 5 2022 9:42AM,247758,21,"NBTY Global, Inc.",,,1
8,Oct 5 2022 9:30AM,247891,16,Sartorius Bioprocess Solutions,,3,
9,Oct 5 2022 9:12AM,247919,10,"Methapharm, Inc.",,,21


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 5 2022 9:57AM,247923,15,"Brookfield Pharmaceuticals, LLC",17,,
1,Oct 5 2022 9:44AM,247898,21,"NBTY Global, Inc.",1,,
2,Oct 5 2022 9:44AM,247760,21,"NBTY Global, Inc.",1,,
3,Oct 5 2022 9:43AM,247903,21,"NBTY Global, Inc.",1,,
4,Oct 5 2022 9:43AM,247677,21,"NBTY Global, Inc.",1,,
5,Oct 5 2022 9:43AM,247921,12,Hush Hush,,1,
6,Oct 5 2022 9:42AM,247802,21,"NBTY Global, Inc.",1,,
7,Oct 5 2022 9:42AM,247758,21,"NBTY Global, Inc.",1,,
8,Oct 5 2022 9:30AM,247891,16,Sartorius Bioprocess Solutions,,3,
9,Oct 5 2022 9:12AM,247919,10,"Methapharm, Inc.",21,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 5 2022 9:57AM,247923,15,"Brookfield Pharmaceuticals, LLC",17,,
1,Oct 5 2022 9:44AM,247898,21,"NBTY Global, Inc.",1,,
2,Oct 5 2022 9:44AM,247760,21,"NBTY Global, Inc.",1,,
3,Oct 5 2022 9:43AM,247903,21,"NBTY Global, Inc.",1,,
4,Oct 5 2022 9:43AM,247677,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 5 2022 9:57AM,247923,15,"Brookfield Pharmaceuticals, LLC",17.0,NaN,NaN
1,Oct 5 2022 9:44AM,247898,21,"NBTY Global, Inc.",1.0,NaN,NaN
2,Oct 5 2022 9:44AM,247760,21,"NBTY Global, Inc.",1.0,NaN,NaN
3,Oct 5 2022 9:43AM,247903,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Oct 5 2022 9:43AM,247677,21,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 5 2022 9:57AM,247923,15,"Brookfield Pharmaceuticals, LLC",17.0,0.0,0.0
1,Oct 5 2022 9:44AM,247898,21,"NBTY Global, Inc.",1.0,0.0,0.0
2,Oct 5 2022 9:44AM,247760,21,"NBTY Global, Inc.",1.0,0.0,0.0
3,Oct 5 2022 9:43AM,247903,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Oct 5 2022 9:43AM,247677,21,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2974847,180.0,27.0,2.0
12,247921,0.0,1.0,0.0
15,743660,67.0,0.0,25.0
16,991554,17.0,3.0,0.0
18,247884,0.0,1.0,0.0
19,991563,8.0,12.0,12.0
20,247875,1.0,0.0,0.0
21,1486798,6.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2974847,180.0,27.0,2.0
1,12,247921,0.0,1.0,0.0
2,15,743660,67.0,0.0,25.0
3,16,991554,17.0,3.0,0.0
4,18,247884,0.0,1.0,0.0
5,19,991563,8.0,12.0,12.0
6,20,247875,1.0,0.0,0.0
7,21,1486798,6.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,180.0,27.0,2.0
1,12,0.0,1.0,0.0
2,15,67.0,0.0,25.0
3,16,17.0,3.0,0.0
4,18,0.0,1.0,0.0
5,19,8.0,12.0,12.0
6,20,1.0,0.0,0.0
7,21,6.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,180.0
1,12,Released,0.0
2,15,Released,67.0
3,16,Released,17.0
4,18,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21
Status,,,,,,,,
Completed,2.0,0.0,25.0,0.0,0.0,12.0,0.0,0.0
Executing,27.0,1.0,0.0,3.0,1.0,12.0,0.0,0.0
Released,180.0,0.0,67.0,17.0,0.0,8.0,1.0,6.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21
0,Completed,2.0,0.0,25.0,0.0,0.0,12.0,0.0,0.0
1,Executing,27.0,1.0,0.0,3.0,1.0,12.0,0.0,0.0
2,Released,180.0,0.0,67.0,17.0,0.0,8.0,1.0,6.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21
0,Completed,2.0,0.0,25.0,0.0,0.0,12.0,0.0,0.0
1,Executing,27.0,1.0,0.0,3.0,1.0,12.0,0.0,0.0
2,Released,180.0,0.0,67.0,17.0,0.0,8.0,1.0,6.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()